In [3]:
def generate_key_matrix(keyword):
    # Generate a Playfair cipher key matrix (5x5 grid) from the keyword
    # Step 1: Remove duplicates, replace 'J' with 'I', and convert to uppercase
    keyword = "".join(dict.fromkeys(keyword.upper().replace('J', 'I')))
    keyword = ''.join(filter(str.isalpha, keyword))  # Keep only alphabetic characters

    # Step 2: Initialize the key matrix with unique characters from the keyword
    key_matrix = []
    for char in keyword:
        if char not in key_matrix:
            key_matrix.append(char)

    # Step 3: Add the remaining letters of the alphabet (excluding 'J')
    for char in 'ABCDEFGHIKLMNOPQRSTUVWXYZ':
        if char not in key_matrix:
            key_matrix.append(char)

    # Step 4: Convert the list into a 5x5 matrix
    matrix = [key_matrix[i:i+5] for i in range(0, 25, 5)]
    return matrix

def preprocess_text(text, encrypt=True):
    # Preprocess the input text: clean and prepare it for encryption/decryption
    text = text.upper().replace('J', 'I')  # Replace 'J' with 'I'
    text = ''.join(filter(str.isalpha, text))  # Remove non-alphabetic characters

    if encrypt:
        # Step 1: Prepare text for encryption by dividing into pairs
        i = 0
        new_text = ''
        while i < len(text):
            a = text[i]
            b = ''
            if i+1 < len(text):  # Check if there's a next character
                b = text[i+1]
            if a == b:
                # Step 2: If two characters in a pair are the same, insert 'X'
                new_text += a + 'X'
                i += 1
            else:
                # Add both characters to the processed text
                new_text += a
                if i+1 < len(text):
                    new_text += b
                else:
                    # Add a filler 'X' if the length is odd
                    new_text += 'X'
                i += 2
        return new_text
    else:
        # For decryption, just clean the text
        return text

def find_position(matrix, char):
    # Find the position (row, column) of a character in the matrix
    for i, row in enumerate(matrix):
        for j, matrix_char in enumerate(row):
            if matrix_char == char:
                return i, j
    return None

def playfair_encrypt(plaintext, keyword):
    # Encrypt plaintext using the Playfair cipher
    matrix = generate_key_matrix(keyword)  # Generate the 5x5 key matrix
    plaintext = preprocess_text(plaintext)  # Preprocess the plaintext
    ciphertext = ''

    # Encrypt the plaintext in pairs of two characters
    for i in range(0, len(plaintext), 2):
        a = plaintext[i]
        b = plaintext[i+1]
        row_a, col_a = find_position(matrix, a)  # Find position of first character
        row_b, col_b = find_position(matrix, b)  # Find position of second character

        if row_a == row_b:
            # If the characters are in the same row, shift columns to the right
            ciphertext += matrix[row_a][(col_a + 1) % 5]
            ciphertext += matrix[row_b][(col_b + 1) % 5]
        elif col_a == col_b:
            # If the characters are in the same column, shift rows down
            ciphertext += matrix[(row_a + 1) % 5][col_a]
            ciphertext += matrix[(row_b + 1) % 5][col_b]
        else:
            # If the characters form a rectangle, swap the columns
            ciphertext += matrix[row_a][col_b]
            ciphertext += matrix[row_b][col_a]

    return ciphertext

def playfair_decrypt(ciphertext, keyword):
    # Decrypt ciphertext using the Playfair cipher
    matrix = generate_key_matrix(keyword)  # Generate the 5x5 key matrix
    ciphertext = preprocess_text(ciphertext, encrypt=False)  # Preprocess the ciphertext
    plaintext = ''

    # Decrypt the ciphertext in pairs of two characters
    for i in range(0, len(ciphertext), 2):
        a = ciphertext[i]
        b = ciphertext[i+1]
        row_a, col_a = find_position(matrix, a)  # Find position of first character
        row_b, col_b = find_position(matrix, b)  # Find position of second character

        if row_a == row_b:
            # If the characters are in the same row, shift columns to the left
            plaintext += matrix[row_a][(col_a - 1) % 5]
            plaintext += matrix[row_b][(col_b - 1) % 5]
        elif col_a == col_b:
            # If the characters are in the same column, shift rows up
            plaintext += matrix[(row_a - 1) % 5][col_a]
            plaintext += matrix[(row_b - 1) % 5][col_b]
        else:
            # If the characters form a rectangle, swap the columns
            plaintext += matrix[row_a][col_b]
            plaintext += matrix[row_b][col_a]

    return plaintext

# Example usage
if __name__ == "__main__":
    keyword = "MONARCHY"  # The keyword for the Playfair cipher
    plaintext = "INSTRUMENTS"  # The plaintext to encrypt

    print("Keyword:", keyword)
    print("Plaintext:", plaintext)

    # Encrypt the plaintext
    ciphertext = playfair_encrypt(plaintext, keyword)
    print("Ciphertext:", ciphertext)

    # Decrypt the ciphertext
    decrypted_text = playfair_decrypt(ciphertext, keyword)
    print("Decrypted Text:", decrypted_text)


Keyword: MONARCHY
Plaintext: INSTRUMENTS
Ciphertext: GATLMZCLRQXA
Decrypted Text: INSTRUMENTSX
